In [1]:
from dask.distributed import Client
import xarray as xr
import gcsfs
import sgkit
from sgkit_bgen.bgen_reader import unpack_variables
xr.set_options(display_style='text')
client = Client()
client

Client Scheduler: tcp://127.0.0.1:8086 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 2 Memory: 21.00 GB


In [2]:
fs = gcsfs.GCSFileSystem()
store = gcsfs.mapping.GCSMap('rs-ukb/prep-data/gt-imputation/ukb_chrXY.zarr', gcs=fs, check=True, create=False)
ds = xr.open_zarr(store)
ds = unpack_variables(ds, dtype='float16')
ds

<xarray.Dataset>
Dimensions:                         (alleles: 2, genotypes: 3, samples: 486443, variants: 45906)
Dimensions without coordinates: alleles, genotypes, samples, variants
Data variables:
    call_genotype_probability_mask  (variants, samples) int8 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    sample_id                       (samples) int32 dask.array<chunksize=(60806,), meta=np.ndarray>
    sample_sex                      (samples) uint8 dask.array<chunksize=(243222,), meta=np.ndarray>
    variant_allele                  (variants, alleles) |S8 dask.array<chunksize=(22953, 1), meta=np.ndarray>
    variant_contig                  (variants) int16 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_contig_name             (variants) |S4 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_id                      (variants) |S15 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_info                    (variants) float64 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_maf                     (variants) float64 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_minor_allele            (variants) |S3 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_position                (variants) int32 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_rsid                    (variants) |S15 dask.array<chunksize=(22953,), meta=np.ndarray>
    call_genotype_probability       (variants, samples, genotypes) float16 dask.array<chunksize=(10432, 11313, 1), meta=np.ndarray>
    call_dosage                     (variants, samples) float16 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    call_dosage_mask                (variants, samples) int8 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
Attributes:
    contig_index:  24
    contig_name:   XY
    contigs:       ['PAR1', 'XY']

In [3]:
cr = ds.call_genotype_probability_mask.mean(dim='samples').compute()
cr.max()

<xarray.DataArray 'call_genotype_probability_mask' ()>
array(0.)

In [3]:
import numpy as np
import dask.array as da
from sgkit.typing import ArrayLike
from xarray import Dataset, DataArray
    
def variant_genotype_counts(ds: Dataset) -> DataArray:
    gti = ds['call_genotype_probability'].argmax(dim='genotypes')
    gti = gti.astype('uint8').expand_dims('genotypes', axis=-1)
    gti = gti == da.arange(ds.dims['genotypes'], dtype='uint8')
    return gti.sum(dim='samples', dtype='uint32')

ds['variant_genotype_counts'] = variant_genotype_counts(ds)[:, [1, 0, 2]]
ds

<xarray.Dataset>
Dimensions:                         (alleles: 2, genotypes: 3, samples: 486443, variants: 45906)
Dimensions without coordinates: alleles, genotypes, samples, variants
Data variables:
    call_genotype_probability_mask  (variants, samples) int8 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    sample_id                       (samples) int32 dask.array<chunksize=(60806,), meta=np.ndarray>
    sample_sex                      (samples) uint8 dask.array<chunksize=(243222,), meta=np.ndarray>
    variant_allele                  (variants, alleles) |S8 dask.array<chunksize=(22953, 1), meta=np.ndarray>
    variant_contig                  (variants) int16 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_contig_name             (variants) |S4 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_id                      (variants) |S15 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_info                    (variants) float64 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_maf                     (variants) float64 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_minor_allele            (variants) |S3 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_position                (variants) int32 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_rsid                    (variants) |S15 dask.array<chunksize=(22953,), meta=np.ndarray>
    call_genotype_probability       (variants, samples, genotypes) float16 dask.array<chunksize=(10432, 11313, 1), meta=np.ndarray>
    call_dosage                     (variants, samples) float16 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    call_dosage_mask                (variants, samples) int8 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    variant_genotype_counts         (variants, genotypes) uint32 dask.array<chunksize=(10432, 3), meta=np.ndarray>
Attributes:
    contig_index:  24
    contig_name:   XY
    contigs:       ['PAR1', 'XY']

In [4]:
ds['dummy'] = xr.DataArray(da.empty((ds.dims['variants'], ds.dims['samples'], 2)), dims=('variants', 'samples', 'ploidy'))
ds['variant_hwe_p_value'] = sgkit.hardy_weinberg_test(ds, genotype_counts='variant_genotype_counts')['variant_hwe_p_value']
ds

<xarray.Dataset>
Dimensions:                         (alleles: 2, genotypes: 3, ploidy: 2, samples: 486443, variants: 45906)
Dimensions without coordinates: alleles, genotypes, ploidy, samples, variants
Data variables:
    call_genotype_probability_mask  (variants, samples) int8 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    sample_id                       (samples) int32 dask.array<chunksize=(60806,), meta=np.ndarray>
    sample_sex                      (samples) uint8 dask.array<chunksize=(243222,), meta=np.ndarray>
    variant_allele                  (variants, alleles) |S8 dask.array<chunksize=(22953, 1), meta=np.ndarray>
    variant_contig                  (variants) int16 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_contig_name             (variants) |S4 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_id                      (variants) |S15 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_info                    (variants) float64 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_maf                     (variants) float64 dask.array<chunksize=(22953,), meta=np.ndarray>
    variant_minor_allele            (variants) |S3 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_position                (variants) int32 dask.array<chunksize=(45906,), meta=np.ndarray>
    variant_rsid                    (variants) |S15 dask.array<chunksize=(22953,), meta=np.ndarray>
    call_genotype_probability       (variants, samples, genotypes) float16 dask.array<chunksize=(10432, 11313, 1), meta=np.ndarray>
    call_dosage                     (variants, samples) float16 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    call_dosage_mask                (variants, samples) int8 dask.array<chunksize=(10432, 11313), meta=np.ndarray>
    variant_genotype_counts         (variants, genotypes) uint32 dask.array<chunksize=(10432, 3), meta=np.ndarray>
    dummy                           (variants, samples, ploidy) float64 dask.array<chunksize=(2186, 2896, 2), meta=np.ndarray>
    variant_hwe_p_value             (variants) float64 dask.array<chunksize=(10432,), meta=np.ndarray>
Attributes:
    contig_index:  24
    contig_name:   XY
    contigs:       ['PAR1', 'XY']

In [ ]:
# This requires about 4GiB per core
p = ds['variant_hwe_p_value'].compute()
p